# Step 1

- Optimizacion Bayesiana de hiperparametros de  rpart
- 10-repeated 5-fold Cross Validation
- trabaja con la clase_ternaria (más lento -puede tardar un día-, pero certero que la binaria)

In [1]:
# Limpiamos el entorno
rm(list = ls())
gc(verbose = FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,627610,33.6,1404611,75.1,994516,53.2
Vcells,1149702,8.8,8388608,64.0,1769393,13.5


In [2]:
# Librerías necesarias
require("data.table")
require("rpart")
require("DiceKriging")
require("mlrMBO")
require("rlist")
require("parallel")

Loading required package: data.table



Loading required package: rpart

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

Loading required package: smoof

Loading required package: checkmate


Attaching package: 'checkmate'


The following object is masked from 'package:DiceKriging':

    checkNames


Loading required package: rlist

Loading required package: parallel



In [3]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 3") # Establezco el Working Directory
#"C:/Users/German/Documents/dmeyf2023/German/Clase 3" #Laptop

In [4]:
# Defino la  Optimizacion Bayesiana
PARAM <- list()

PARAM$home <- "C:/Users/German/Desktop/dmeyf2023/German/Clase 3"

PARAM$experimento <- "HT3220_alternativo"

# Aqui van las 10 semillas que hacen el 10-repeated
#  si se pone una sola semilla, se esta haciendo solo 5-fold xval
PARAM$semilla_azar <- c(279511, 279523, 279541, 279551, 279571,
279581, 279591, 279592, 279593, 279594) #Agrego 5 más para llegar a las 10

In [5]:
# Cargamos el dataset
#dataset <- fread("competencia_01.csv")

# Nos quedamos solo con el 202103
#dataset <- dataset[foto_mes == 202103]

In [6]:
# folds del cross validation
PARAM$xval_folds <- 5

# cantidad de iteraciones de la Optimizacion Bayesiana
PARAM$BO_iter <- 300

# la letra L al final de 1L significa ENTERO
PARAM$hs <- makeParamSet(
  makeNumericParam("cp", lower = -1, upper = 0.1),
  makeIntegerParam("minsplit", lower = 1L, upper = 8000L),
  makeIntegerParam("minbucket", lower = 1L, upper = 4000L),
  makeIntegerParam("maxdepth", lower = 3L, upper = 20L),
  forbidden = quote(minbucket > 0.5 * minsplit)
)
# minbuket NO PUEDE ser mayor que la mitad de minsplit


# este valor debe ser 1 si se utiliza Windows
PARAM$cores <- 1

# Step 2: graba a un archivo los componentes de lista para el primer registro, escribe antes los titulos

In [7]:
loguear <- function(reg, arch = NA, folder = "./work/", ext = ".txt",
                    verbose = TRUE) {
  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  # Escribo los titulos
  if (!file.exists(archivo)) {
    linea <- paste0(
      "fecha\t",
      paste(list.names(reg), collapse = "\t"), "\n"
    )

    cat(linea, file = archivo)
  }

  # la fecha y hora
  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t",
    gsub(", ", "\t", toString(reg)), "\n"
  )

  # grabo al archivo
  cat(linea, file = archivo, append = TRUE)

  # imprimo por pantalla
  if (verbose) cat(linea)
}

- Particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa 
- particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30
- particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones


In [8]:
particionar <- function(data, division, agrupa = "", campo = "fold",
                        start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(
    function(x, y) {
      rep(y, x)
    }, division,
    seq(from = start, length.out = length(division))
  ))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}

In [9]:
# fold_test  tiene el numero de fold que voy a usar para testear,
#  entreno en el resto de los folds
# param tiene los hiperparametros del arbol

ArbolSimple <- function(fold_test, data, param) {
  # genero el modelo
  # entreno en todo MENOS el fold_test que uso para testing
  modelo <- rpart("clase_ternaria ~ .",
    data = data[fold != fold_test, ],
    xval = 0,
    control = param
  )

  # aplico el modelo a los datos de testing
  # aplico el modelo sobre los datos de testing
  # quiero que me devuelva probabilidades
  prediccion <- predict(modelo,
    data[fold == fold_test, ],
    type = "prob"
  )

  # esta es la probabilidad de baja
  prob_baja2 <- prediccion[, "BAJA+2"]

  # calculo la ganancia
  ganancia_testing <- data[fold == fold_test][
    prob_baja2 > 1 / 40,
    sum(ifelse(clase_ternaria == "BAJA+2",
      273000, -7000
    ))
  ]

  # esta es la ganancia sobre el fold de testing, NO esta normalizada
  return(ganancia_testing)
}

In [10]:
ArbolesCrossValidation <- function(data, param, qfolds, pagrupa, semilla) {
  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos
  divi <- rep(1, qfolds)

  # particiono en dataset en folds
  particionar(data, divi, seed = semilla, agrupa = pagrupa)

  ganancias <- mcmapply(ArbolSimple,
    seq(qfolds), # 1 2 3 4 5
    MoreArgs = list(data, param),
    SIMPLIFY = FALSE,
    mc.cores = PARAM$cores
  )

  data[, fold := NULL]

  ganancia_xval <- sum(unlist(ganancias))

  return(ganancia_xval)
}

In [11]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # param= x los hiperparametros del arbol
  # qfolds= PARAM$xval_folds  la cantidad de folds
  gans <- c()
  for (semilla in PARAM$semilla_azar)
  {
    ganancia <- ArbolesCrossValidation(dataset,
      param = x,
      qfolds = PARAM$xval_folds,
      pagrupa = "clase_ternaria",
      semilla = semilla
    )

    gans <- c(gans, ganancia)
  }
  # imprimo el vector con las ganancias de xval
  # para que se aprecia la variabilidad aun con 5-fold
  cat(gans, "\n")

  # logueo
  xx <- x
  xx$xval_repeats <- length(PARAM$semilla_azar)
  xx$xval_folds <- PARAM$xval_folds
  xx$ganancia <- mean(gans)
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch = archivo_log)

  return(mean(gans))
}

# Step 3: Aqui empieza el programa

In [12]:
# Establezco el Working Directory inicial
setwd(PARAM$home)

# cargo los datos
dataset <- fread("competencia_01_alternativo.csv")
# entreno en 202103
dataset <- dataset[foto_mes == 202103]

In [13]:
# creo la carpeta donde va el experimento
#  HT  representa  Hiperparameter Tuning
dir.create("./exp/", showWarnings = FALSE)

dir.create(paste0("./exp/", PARAM$experimento, "/"),
  showWarnings = FALSE
)

In [14]:
# Establezco el Working Directory DEL EXPERIMENTO
setwd(paste0("./exp/", PARAM$experimento, "/"))

In [15]:
archivo_log <- "BO_log.txt"
archivo_BO <- "bayesiana.RDATA"

# leo si ya existe el log
#  para retomar en caso que se se corte el programa
GLOBAL_iteracion <- 0

if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
}

# Step 4:"Aqui comienza la configuracion de la Bayesian Optimization"

In [16]:
funcion_optimizar <- EstimarGanancia

In [17]:
configureMlr(show.learner.output = FALSE)

In [18]:
# configuro la busqueda bayesiana,
#  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
# minimize= FALSE estoy Maximizando la ganancia
obj.fun <- makeSingleObjectiveFunction(
  fn = funcion_optimizar,
  minimize = FALSE,
  noisy = TRUE,
  par.set = PARAM$hs,
  has.simple.signature = FALSE
)

In [19]:
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600,
  save.file.path = archivo_BO
)

In [20]:
ctrl <- setMBOControlTermination(ctrl, iters = PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())


In [21]:
surr.km <- makeLearner("regr.km",
  predict.type = "se",
  covtype = "matern3_2", control = list(trace = TRUE)
)

In [22]:
# inicio la optimizacion bayesiana
if (!file.exists(archivo_BO)) {
  run <- mbo(
    fun = obj.fun,
    learner = surr.km,
    control = ctrl
  )
} else {
  run <- mboContinue(archivo_BO)
}
# retomo en caso que ya exista

Computing y column(s) for design. Not provided.



70595000 67144000 67494000 66339000 69881000 69265000 67963000 65947000 63133000 70567000 
20230905 110017	-0.284313363989349	3345	745	17	10	5	67832800	1
51520000 50064000 49791000 53704000 55426000 51051000 52430000 50456000 50694000 51541000 
20230905 110707	-0.452143641882867	6974	890	20	10	5	51667700	2
62510000 61922000 60522000 60109000 61656000 59514000 60550000 60592000 58002000 63644000 
20230905 111115	-0.145418120472459	2952	1306	4	10	5	60902100	3
51520000 50246000 49791000 53704000 55426000 51051000 52430000 50848000 51590000 51464000 
20230905 111546	-0.661265737036592	7174	1534	5	10	5	51807000	4
67067000 70462000 72597000 70791000 73066000 72373000 72674000 74137000 66850000 71309000 
20230905 112227	-0.602451440013829	1918	455	9	10	5	71132600	5
51520000 52129000 49791000 53704000 55426000 51051000 55125000 51198000 54831000 51422000 
20230905 112852	-0.839975238118495	7793	2067	16	10	5	52619700	6
52528000 51702000 49168000 53809000 56707000 54012000 54789000 50708000 5020

[mbo] 0: cp=-0.284; minsplit=3345; minbucket=745; maxdepth=17 : y = 6.78e+07 : 500.5 secs : initdesign

[mbo] 0: cp=-0.452; minsplit=6974; minbucket=890; maxdepth=20 : y = 5.17e+07 : 410.4 secs : initdesign

[mbo] 0: cp=-0.145; minsplit=2952; minbucket=1306; maxdepth=4 : y = 6.09e+07 : 248.3 secs : initdesign

[mbo] 0: cp=-0.661; minsplit=7174; minbucket=1534; maxdepth=5 : y = 5.18e+07 : 270.8 secs : initdesign

[mbo] 0: cp=-0.602; minsplit=1918; minbucket=455; maxdepth=9 : y = 7.11e+07 : 401.1 secs : initdesign

[mbo] 0: cp=-0.84; minsplit=7793; minbucket=2067; maxdepth=16 : y = 5.26e+07 : 384.7 secs : initdesign

[mbo] 0: cp=-0.195; minsplit=6321; minbucket=98; maxdepth=14 : y = 5.27e+07 : 553.4 secs : initdesign

[mbo] 0: cp=-0.357; minsplit=5633; minbucket=951; maxdepth=19 : y = 5.6e+07 : 412.0 secs : initdesign

[mbo] 0: cp=-0.49; minsplit=6751; minbucket=1351; maxdepth=18 : y = 5.27e+07 : 368.7 secs : initdesign

[mbo] 0: cp=-0.211; minsplit=7150; minbucket=154; maxdepth=13 : y =

30254000 39193000 29078000 33880000 34433000 29617000 34258000 35077000 36323000 30744000 
20230905 123943	-0.555082874375618	16	6	17	10	5	33285700	17


[mbo] 1: cp=-0.555; minsplit=16; minbucket=6; maxdepth=17 : y = 3.33e+07 : 762.6 secs : infill_ei

Saved the current state after iteration 2 in the file bayesiana.RDATA.



...

[mbo] 298: cp=-0.783; minsplit=5231; minbucket=2613; maxdepth=20 : y = 5.9e+07 : 338.8 secs : infill_ei



59619000 59010000 54292000 59122000 58835000 58499000 58996000 59626000 54607000 54838000 
20230906 200349	-0.999825408687272	5477	2735	4	10	5	57744400	315


[mbo] 299: cp=-1; minsplit=5477; minbucket=2735; maxdepth=4 : y = 5.77e+07 : 216.4 secs : infill_ei



67970000 64925000 66332000 66269000 69664000 63154000 66689000 65135000 65107000 67487000 
20230906 201032	-0.398059665473806	2534	1267	20	10	5	66273200	316


[mbo] 300: cp=-0.398; minsplit=2534; minbucket=1267; maxdepth=20 : y = 6.63e+07 : 398.5 secs : infill_ei

Saved the final state in the file bayesiana.RDATA

Saved the final state in the file bayesiana.RDATA

